In [1]:
from google.colab import drive
drive.mount("/content/drive")
modelPath = "/content/drive/My Drive/ambulance_prediction"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import applications
from keras import optimizers
from imutils import paths
import os
import numpy as np

Using TensorFlow backend.


In [3]:
img_width, img_height = 150, 150

train_data_dir = "/content/drive/My Drive/ambulance_prediction/data/train"
validation_data_dir = "/content/drive/My Drive/ambulance_prediction/data/validation"
top_model_weights_path = "/content/drive/My Drive/ambulance_prediction"

nb_train_samples = len(list(paths.list_images(train_data_dir)))
nb_validation_samples = len(list(paths.list_images(validation_data_dir)))
epochs = 50
batch_size = 16

In [8]:
# build the VGG16 network
base_model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

Model loaded.


In [10]:
from keras.models import Model
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(os.path.join(top_model_weights_path, 'first_try_pretrained.h5'))

# add the model on top of the convolutional base
model = Model(input= base_model.input, output = top_model(base_model.output))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  


In [12]:
from keras import optimizers
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
  layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

H = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Found 1106 images belonging to 2 classes.
Found 294 images belonging to 2 classes.
Epoch 1/50
 3/69 [>.............................] - ETA: 6:26 - loss: 0.2825 - accuracy: 0.9583

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


54/69 [======================>.......] - ETA: 1:07 - loss: 0.3438 - accuracy: 0.9549

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


69/69 [==============================] - 378s 5s/step - loss: 0.3093 - accuracy: 0.9583 - val_loss: 0.6888 - val_accuracy: 0.8958
Epoch 2/50
69/69 [==============================] - 320s 5s/step - loss: 0.3379 - accuracy: 0.9535 - val_loss: 0.0923 - val_accuracy: 0.8993
Epoch 3/50
69/69 [==============================] - 323s 5s/step - loss: 0.4446 - accuracy: 0.9495 - val_loss: 1.5125 - val_accuracy: 0.8921
Epoch 4/50
69/69 [==============================] - 321s 5s/step - loss: 0.4612 - accuracy: 0.9450 - val_loss: 2.7234 - val_accuracy: 0.8993
Epoch 5/50
69/69 [==============================] - 326s 5s/step - loss: 0.4506 - accuracy: 0.9495 - val_loss: 2.6497 - val_accuracy: 0.9029
Epoch 6/50
60/69 [=========================>....] - ETA: 33s - loss: 0.4345 - accuracy: 0.9521

In [ ]:
model.save(os.path.join(top_model_weights_path, 'first_try_pretrained_chk.h5'))

In [ ]:
print(H.history.keys())
#ploting the accuracy
plt.plot(H.history('accuracy'))
plt.plot(H.history('val_accuracy'))
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()
#plotting the loss
plt.plot(H.history('loss'))
plt.plot(H.history('val_loss'))
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()